In [1]:
import sys
import platform
import torch
import torchvinecopulib as tvc
import pyvinecopulib as pvc
import numpy as np
import pandas as pd
import warnings
import random
from collections import defaultdict
from torchvinecopulib.util import _EPS

warnings.filterwarnings("ignore", category=RuntimeWarning)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MTD_KDE = "tll"
NUM_OBS = 20000
# DEVICE = "cpu"
print("Python:", sys.version.replace("\n", " "))
print("Platform:", platform.platform())
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Using torchvinecopulib version:", tvc.__version__)
print("Using pyvinecopulib version:", pvc.__version__)

Python: 3.13.2 (main, Mar 17 2025, 21:02:54) [Clang 20.1.0 ]
Platform: Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35
PyTorch: 2.7.1+cu128
CUDA available: True
Using torchvinecopulib version: 1.1.2
Using pyvinecopulib version: 0.7.3


# Bicop Comparison

> various grid size


In [ ]:
num_seed = 66
fit_ctrl = pvc.FitControlsBicop(
    family_set=[pvc.BicopFamily.tll], num_threads=torch.get_num_threads()
)

res_bcp = defaultdict(list)
# * grid for eval
axis = torch.linspace(0.01, 0.99, 100)
V_grid_tensor = torch.cartesian_prod(axis, axis).view(-1, 2)
V_grid_numpy = V_grid_tensor.cpu().numpy()
for seed in range(num_seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # * random true bicop
    rho = random.uniform(0.1, 0.9)
    nu = random.uniform(2.0, 10.0)
    bcp_true = pvc.Bicop(family=pvc.BicopFamily.student, parameters=np.array([[rho], [nu]]))
    # * obs from true bicop
    V_bcp_numpy = bcp_true.simulate(NUM_OBS, qrng=True)
    V_bcp_tensor = torch.from_numpy(V_bcp_numpy)
    for num_step_grid in [16, 32, 64, 128, 256]:
        tll_pvc = pvc.Bicop.from_data(data=V_bcp_numpy, controls=fit_ctrl)
        tll_tvc = tvc.BiCop(num_step_grid=num_step_grid)
        tll_tvc.fit(obs=V_bcp_tensor, mtd_kde=MTD_KDE)
        tll_tvc_cuda = tvc.BiCop(num_step_grid=num_step_grid).cuda()
        tll_tvc_cuda.fit(obs=V_bcp_tensor.cuda(), mtd_kde=MTD_KDE)
        # ! pdf
        ref = bcp_true.pdf(V_grid_numpy)
        res_bcp[("pdf", "pvc", num_step_grid)].append(
            pd.Series(tll_pvc.pdf(V_grid_numpy) - ref).abs()
        )
        res_bcp[("pdf", "tvc", num_step_grid)].append(
            pd.Series(tll_tvc.pdf(obs=V_grid_tensor).flatten().numpy() - ref).abs()
        )
        res_bcp[("pdf", "tvc_cuda", num_step_grid)].append(
            pd.Series(
                tll_tvc_cuda.pdf(obs=V_grid_tensor.cuda()).flatten().cpu().numpy() - ref
            ).abs()
        )
        # ! h-function
        ref = bcp_true.hfunc1(V_grid_numpy)
        res_bcp[("hfunc", "pvc", num_step_grid)].append(
            pd.Series(tll_pvc.hfunc1(V_grid_numpy) - ref).abs()
        )
        res_bcp[("hfunc", "tvc", num_step_grid)].append(
            pd.Series(tll_tvc.hfunc_l(obs=V_grid_tensor).flatten().numpy() - ref).abs()
        )
        res_bcp[("hfunc", "tvc_cuda", num_step_grid)].append(
            pd.Series(
                tll_tvc_cuda.hfunc_l(obs=V_grid_tensor.cuda()).flatten().cpu().numpy() - ref
            ).abs()
        )
        # ! h-inverse
        ref = bcp_true.hinv1(V_grid_numpy)
        res_bcp[("hinv", "pvc", num_step_grid)].append(
            pd.Series(tll_pvc.hinv1(V_grid_numpy) - ref).abs()
        )
        res_bcp[("hinv", "tvc", num_step_grid)].append(
            pd.Series(tll_tvc.hinv_l(obs=V_grid_tensor).flatten().numpy() - ref).abs()
        )
        res_bcp[("hinv", "tvc_cuda", num_step_grid)].append(
            pd.Series(
                tll_tvc_cuda.hinv_l(obs=V_grid_tensor.cuda()).flatten().cpu().numpy() - ref
            ).abs()
        )

In [6]:
import pandas as pd
from collections import defaultdict

# Assuming res_bcp is populated as in the provided script

# Step 1: Aggregate across seeds
grouped_means = defaultdict(list)
grouped_stds = defaultdict(list)

# Extract the keys and aggregate the values
for key, series_list in res_bcp.items():
    # key = (metric, device, num_step_grid)
    num_step_grid = key[2]
    stacked = pd.concat(series_list, axis=0)  # Each column is one seed
    # Store overall mean and std per num_step_grid
    grouped_means[(key[0], key[1], num_step_grid)] = stacked.mean(axis=0)
    grouped_stds[(key[0], key[1], num_step_grid)] = stacked.std(axis=0)

# Step 2: Convert to DataFrames for summarization
df_mean = pd.DataFrame(
    [
        {"metric": k[0], "device": k[1], "num_step_grid": k[2], "mean_abs_error": v}
        for k, v in grouped_means.items()
    ]
)

df_std = pd.DataFrame(
    [
        {"metric": k[0], "device": k[1], "num_step_grid": k[2], "std_abs_error": v}
        for k, v in grouped_stds.items()
    ]
)
# Convert the flat df_mean structure to a wide format with MultiIndex columns (metric, device)
df_mean_pivoted = df_mean.pivot(
    index="num_step_grid", columns=["metric", "device"], values="mean_abs_error"
)
df_std_pivoted = df_std.pivot(
    index="num_step_grid", columns=["metric", "device"], values="std_abs_error"
)

In [8]:
df_mean_pivoted

metric              pdf                         hfunc                      \
device              pvc       tvc  tvc_cuda       pvc       tvc  tvc_cuda   
num_step_grid                                                               
16             0.022852  0.316856  0.316856  0.003151  0.045896  0.045896   
32             0.022852  0.068554  0.068554  0.003151  0.023756  0.023756   
64             0.022852  0.027740  0.027740  0.003151  0.012075  0.012075   
128            0.022852  0.022657  0.022657  0.003151  0.006454  0.006454   
256            0.022852  0.022740  0.022740  0.003151  0.004110  0.004110   

metric             hinv                      
device              pvc       tvc  tvc_cuda  
num_step_grid                                
16             0.003141  0.046079  0.046079  
32             0.003141  0.023771  0.023771  
64             0.003141  0.012033  0.012033  
128            0.003141  0.006405  0.006405  
256            0.003141  0.004074  0.004074

In [9]:
df_std_pivoted

metric              pdf                         hfunc                      \
device              pvc       tvc  tvc_cuda       pvc       tvc  tvc_cuda   
num_step_grid                                                               
16             0.059667  5.560241  5.560241  0.003352  0.059511  0.059511   
32             0.059667  1.831398  1.831398  0.003352  0.033411  0.033411   
64             0.059667  0.304828  0.304828  0.003352  0.015503  0.015503   
128            0.059667  0.071467  0.071467  0.003352  0.006919  0.006919   
256            0.059667  0.067312  0.067312  0.003352  0.004319  0.004319   

metric             hinv                      
device              pvc       tvc  tvc_cuda  
num_step_grid                                
16             0.002703  0.065626  0.065626  
32             0.002703  0.039785  0.039785  
64             0.002703  0.019628  0.019628  
128            0.002703  0.009502  0.009502  
256            0.002703  0.005094  0.005094

# vinecop
